In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale

In [2]:
path = 'data/'

# Meat Consumption

http://www.fao.org/faostat/en/?#data/FBS/visualize

In [40]:
meat_consumption_path = path + 'FoodBalanceSheets_E_Europe_1.csv'
meat_consumption = pd.read_csv(meat_consumption_path, low_memory=False)

In [41]:
years_in_dataset = range(1961, 2014)
years_to_keep = range(2004, 2014) # 2013 is the last year available in the dataset
years_to_drop = range(1961, 2004) # we don't use data before 2004

In [42]:
columns_to_drop = ['Y' + str(x) + 'F' for x in years_in_dataset] # These columns doesn't contain relevant data
columns_to_drop += ['Y' + str(x) for x in years_to_drop]
meat_consumption = meat_consumption.drop(columns=columns_to_drop)

In [43]:
rename_columns = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit'] + [x for x in years_to_keep]

In [38]:
rename_columns

['Area Code',
 'Area',
 'Item Code',
 'Item',
 'Element Code',
 'Element',
 'Unit',
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013]

In [44]:
meat_consumption.columns = rename_columns

In [47]:
# meat_consumption.Item.unique()
meat_related_item = ['Bovine Meat', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Meat, Other']

In [48]:
meat_consumption = meat_consumption[meat_consumption.Item.isin(meat_related_item)]

In [50]:
europe = [
    "Austria", 
    "Belgium", 
    "Bulgaria", 
    "Croatia", 
    "Republic of Cyprus", 
    "Czech Republic", 
    "Denmark", 
    "Estonia", 
    "Finland", 
    "France", 
    "Germany", 
    "Greece", 
    "Hungary", 
    "Ireland", 
    "Italy", 
    "Latvia", 
    "Lithuania", 
    "Luxembourg", 
    "Malta", 
    "Netherlands", 
    "Poland", 
    "Portugal", 
    "Romania", 
    "Slovakia", 
    "Slovenia", 
    "Spain", 
    "Sweden", 
    "United Kingdom"
]

In [51]:
meat_consumption = meat_consumption[meat_consumption.Area.isin(europe)]

In [54]:
meat_consumption_melted = pd.melt(meat_consumption, id_vars=["Area Code", "Area", "Item Code", "Item", 'Element Code' , 'Element', 'Unit'], 
             value_vars=years_to_keep)

In [55]:
meat_consumption_melted.columns

Index(['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element',
       'Unit', 'variable', 'value'],
      dtype='object')

In [56]:
new_columns = ["Area Code", "Area", "Item Code", "Item", 'Element Code' , 'Element', 'Unit', 'years', 'value']

In [57]:
meat_consumption_melted.columns = new_columns

In [80]:
meat_consumption_melted = meat_consumption_melted[meat_consumption_melted['Element'] == 'Domestic supply quantity']

In [81]:
meat_consumption_melted.Item.unique()

array(['Bovine Meat', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat',
       'Meat, Other'], dtype=object)

## All meats

In [82]:
meat_consumption_melted.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,years,value
4,11,Austria,2731,Bovine Meat,5301,Domestic supply quantity,1000 tonnes,2004,149.0
14,11,Austria,2732,Mutton & Goat Meat,5301,Domestic supply quantity,1000 tonnes,2004,9.0
23,11,Austria,2733,Pigmeat,5301,Domestic supply quantity,1000 tonnes,2004,466.0
34,11,Austria,2734,Poultry Meat,5301,Domestic supply quantity,1000 tonnes,2004,151.0
45,11,Austria,2735,"Meat, Other",5301,Domestic supply quantity,1000 tonnes,2004,8.0


In [112]:
meat_consumption_melted = meat_consumption_melted.groupby(['years', 'Area'])['value'].sum().unstack(level=0)

meat_consumption_melted['Area'] = meat_consumption_melted.index

meat_consumption_melted = pd.melt(meat_consumption_melted, id_vars=['Area'], value_vars=years_to_keep)

In [113]:
meat_consumption_melted.head()

,Area,years,value
0,Austria,2004,783.0
1,Belgium,2004,837.0
2,Bulgaria,2004,391.0
3,Croatia,2004,216.0
4,Denmark,2004,397.0


In [114]:
fig = px.line(meat_consumption_melted, x="years", y='value', color='Area')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='All meats consumption per 1000 tons')
fig.show()

In [116]:
meat_consumption_melted.value.describe()

count     260.000000
mean     1592.296154
std      2060.610499
min        39.000000
25%       274.000000
50%       737.000000
75%      1421.500000
max      7380.000000
Name: value, dtype: float64

In [119]:
min_val = meat_consumption_melted.value.min()
max_val = meat_consumption_melted.value.max()

fig = px.choropleth(meat_consumption_melted, 
                    locations="Area", 
                    scope="europe", 
                    locationmode = 'country names', 
                    color='value', 
                    hover_name="Area", 
                    animation_frame="years", 
                    range_color=[min_val, 6500])
fig.show()

## Normalize all

In [122]:
meat_consumption_melted_normalized = meat_consumption_melted.copy()

In [143]:
# normalize value column for each country
meat_consumption_melted_normalized['value'] = meat_consumption_melted_normalized.groupby(['Area']).value.transform(lambda x: minmax_scale(x.astype(float)))

In [148]:
fig = px.line(meat_consumption_melted_normalized, x="years", y='value', color='Area')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='All meats consumption per 1000 tons (normalized per country)')
fig.show()

In [150]:
min_val = meat_consumption_melted_normalized.value.min()
max_val = meat_consumption_melted_normalized.value.max()

fig = px.choropleth(meat_consumption_melted_normalized, 
                    locations="Area", 
                    scope="europe", 
                    locationmode = 'country names', 
                    color='value', 
                    hover_name="Area", 
                    animation_frame="years", 
                    range_color=[min_val, max_val])
fig.show()

## Poultry Meat 

In [61]:
poultry_consumption = meat_consumption_melted[meat_consumption_melted['Item'] == 'Poultry Meat']

In [68]:
# poultry_consumption = poultry_consumption[poultry_consumption['Element'] == 'Domestic supply quantity']

In [83]:
fig = px.line(poultry_consumption, x="years", y='value', color='Area')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='Poultry consumption in 1000 tons')
fig.show()

In [72]:
poultry_consumption.value.describe()

count     260.000000
mean      418.307692
std       542.517147
min         8.000000
25%        76.000000
50%       151.500000
75%       421.250000
max      2091.000000
Name: value, dtype: float64

In [78]:
min_val = poultry_consumption.value.min()
max_val = poultry_consumption.value.max()

fig = px.choropleth(poultry_consumption, 
                    locations="Area", 
                    scope="europe", 
                    locationmode = 'country names', 
                    color='value', 
                    hover_name="Area", 
                    animation_frame="years", 
                    range_color=[min_val, max_val])
fig.show()